In [ ]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from cuml.ensemble import RandomForestClassifier as cuRFC
from cuml.datasets.classification import make_classification as make_classification_cu
from cuml.preprocessing.model_selection import train_test_split as train_test_split_cu
import cudf
from cuml.metrics import accuracy_score as accuracy_score_cu

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
#Есть в последних версиях
from cuml.experimental.explainer import KernelExplainer as cuKE

cuML - это по сути sklearn, но только на GPU, хотя и не все алгоритмы в нем реализованы, как и в остальных библиотеках экосистемы rapids.

Простой пример: сгенерируем датасет для бинарной классификации, разделим стандартно на обучающее и тестовое множесто, обучим RandomForest. 

P.S. тут не про подбор параметров и наилучший способ обучения моделей =)

In [ ]:
# параметры 
n_samples = 10000000
n_features = 20
n_classes = 2

# random forest depth and size
n_estimators = 30
max_depth = 15

In [ ]:
%%time 
X, y = make_classification(n_classes=n_classes, n_features=n_features, n_samples=n_samples, random_state=0, class_sep=0.7)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
%%time 
X_cu, y_cu = make_classification_cu(n_classes=n_classes, n_features=n_features, n_samples=n_samples, random_state=0, class_sep=0.7)
X_train_cu, X_test_cu, y_train_cu, y_test_cu = train_test_split_cu(X_cu, y_cu, random_state=0)

In [ ]:
X_cu.device

Прирост есть, да и данные сразу на GPU.

А что с обучением моделей? (даже при первом запуске все хорошо, но при повтороном еще лучше, надо "прогревать")

In [ ]:
import os
print(f'Количество ядер CPU: {os.cpu_count()}')

In [ ]:
import os
mem_bytes = os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')  # e.g. 4015976448
mem_gib = mem_bytes/(1024.**3)
print(mem_gib)

In [ ]:
%time clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=0).fit(X_train, y_train)

In [ ]:
%time clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=0, n_jobs=-1).fit(X_train, y_train)

In [ ]:
%time print(f'Train: {accuracy_score(y_train, clf.predict(X_train)):.2f}, Test: {accuracy_score(y_test, clf.predict(X_test)):.2f}')

In [ ]:
%time clf_cu = cuRFC(n_estimators=n_estimators, max_depth=max_depth, random_state=0).fit(X_train_cu, y_train_cu)

In [ ]:
%time print(f'''Train: {accuracy_score_cu(y_train_cu, clf_cu.predict(X_train_cu)):.2f},\
                Test: {accuracy_score_cu(y_test_cu, clf_cu.predict(X_test_cu)):.2f}''')

Как мы видим, обучение проходит хорошо и довольно быстро, намного быстрее, чем на CPU, даже при большом количестве ядер (не говоря уже об одном)

Давайте еще посмотрим, что были у нас за данные=)

In [ ]:
from cuml.manifold.umap import UMAP as cuUMAP
import umap

In [ ]:
%%time
umap_cpu = umap.UMAP(n_neighbors=30, n_components=2).fit(X)
umap_cpu_embeddings = umap_cpu.transform(X)

In [ ]:
a, b = zip(*umap_cpu_embeddings.tolist())
plt.scatter(a, b, c=y)

In [ ]:
%%time
umap_gpu = cuUMAP(n_neighbors=30, n_components=2).fit(X)
umap_gpu_embeddings = umap_gpu.transform(X)

In [ ]:
a, b = zip(*umap_gpu_embeddings.tolist())
plt.scatter(a, b, c=y)

### Сохранение и загурзка объектов (обучение на одном GPU)

In [ ]:
from joblib import dump, load

dump(clf_cu, 'RF.model')
model = load('RF.model')
model

Тажкже можно через pickle

In [ ]:
import pickle

pickle.dump(clf_cu, open("RF.pkl", "wb"))
model = pickle.load(open("RF.pkl", "rb"))
model

Если модель обучалась на нескольких GPU, то придется сделать одно действие:

In [ ]:
single_gpu_model = model.get_combined_model()
pickle.dump(single_gpu_model, open("single_gpu_model.pkl", "wb"))

### Так, как сохранять модель, обученную на нескольких GPU показали, а обучать то как?

In [ ]:
from cuml.dask.ensemble import RandomForestClassifier as daskRFC
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from cuml.dask.common import utils as dask_utils
import dask_cudf

In [ ]:
import subprocess
cmd = "hostname --all-ip-addresses"
process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
IPADDR = str(output.decode()).split()[0]

In [ ]:
cluster = LocalCUDACluster(ip=IPADDR, local_directory='/data/kuznetsov2-md/TEMP/', CUDA_VISIBLE_DEVICES='0,1')
client = Client(cluster)

In [ ]:
client

In [ ]:
workers = client.has_what().keys()
n_workers = len(workers)

In [ ]:
workers

In [ ]:
X_cu, y_cu = make_classification_cu(n_classes=n_classes, n_features=n_features, n_samples=n_samples, random_state=0, class_sep=0.7)

In [ ]:
# Сначала нужно конвертировать данные в cudf, типы float32 и int32, на 64 будет ругаться=)
X_train_cudf = cudf.DataFrame(X_cu)
y_train_cu = y_cu.astype('int32')
y_train_cudf = cudf.Series(y_train_cu)

# Разбиваем данные на партиции (обычно сколько гпу, столькои  партиций)
X_train_dask = dask_cudf.from_cudf(X_train_cudf, npartitions=n_workers).persist()
y_train_dask = dask_cudf.from_cudf(y_train_cudf, npartitions=n_workers).persist()

In [ ]:
%%time 
rf = daskRFC(n_estimators=n_estimators, max_depth=max_depth, random_state=0,
                   ignore_empty_partitions=True, client=client).fit(X_train_dask, y_train_dask)

Сразу после обучения распределенной модели может не получится ее сохранить, метод get_combined_model вернет путой объект. Если так происходит, то можно 1 раз вызвать метод predict, тогда модель уже будет сериализованным объектом и нормально сохранится

In [ ]:
rf.predict(X_train_dask)

In [ ]:
single_gpu_model = rf.get_combined_model()
pickle.dump(single_gpu_model, open("single_gpu_model.pkl", "wb"))
model = pickle.load(open("single_gpu_model.pkl", "rb"))
model